In [1]:
import sqlite3
sqliteConnection = sqlite3.connect('sql.db')

In [4]:
cursor = sqliteConnection.cursor()

In [17]:
"""
points, segments
between, sameside
parallel, collinear, midpoint, congruent, similar, perpendicular, concyclic
angle, angle_sum, length, length_ratio
"""
def create_table(name, n_points):
  query = ""
  for i in range(n_points):
    query += f"p{i} CHAR(10) NOT NULL,"
  for i in range(n_points):
    query += f"FOREIGN KEY(p{i}) REFERENCES points(name),"
  query = query[:-1]
  query = f"CREATE TABLE {name} ({query})"
  print(query)
  cursor.execute(query)
  return query
points = """ CREATE TABLE points (
            name CHAR(10) PRIMARY KEY NOT NULL
        ); """
cursor.execute(points)
create_table("segments", 2)
create_table("`between`", 3)
create_table("sameside", 3)
create_table("parallel", 4)
create_table("collinear", 3)
create_table("midpoint", 2)
create_table("congruent", 6)
create_table("similar", 6)
create_table("perpendicular", 4)
create_table("concyclic", 4)

CREATE TABLE `between` (p0 CHAR(10) NOT NULL,p1 CHAR(10) NOT NULL,p2 CHAR(10) NOT NULL,FOREIGN KEY(p0) REFERENCES points(name),FOREIGN KEY(p1) REFERENCES points(name),FOREIGN KEY(p2) REFERENCES points(name))
CREATE TABLE sameside (p0 CHAR(10) NOT NULL,p1 CHAR(10) NOT NULL,p2 CHAR(10) NOT NULL,FOREIGN KEY(p0) REFERENCES points(name),FOREIGN KEY(p1) REFERENCES points(name),FOREIGN KEY(p2) REFERENCES points(name))
CREATE TABLE parallel (p0 CHAR(10) NOT NULL,p1 CHAR(10) NOT NULL,p2 CHAR(10) NOT NULL,p3 CHAR(10) NOT NULL,FOREIGN KEY(p0) REFERENCES points(name),FOREIGN KEY(p1) REFERENCES points(name),FOREIGN KEY(p2) REFERENCES points(name),FOREIGN KEY(p3) REFERENCES points(name))
CREATE TABLE collinear (p0 CHAR(10) NOT NULL,p1 CHAR(10) NOT NULL,p2 CHAR(10) NOT NULL,FOREIGN KEY(p0) REFERENCES points(name),FOREIGN KEY(p1) REFERENCES points(name),FOREIGN KEY(p2) REFERENCES points(name))
CREATE TABLE midpoint (p0 CHAR(10) NOT NULL,p1 CHAR(10) NOT NULL,FOREIGN KEY(p0) REFERENCES points(name),FORE

'CREATE TABLE concyclic (p0 CHAR(10) NOT NULL,p1 CHAR(10) NOT NULL,p2 CHAR(10) NOT NULL,p3 CHAR(10) NOT NULL,FOREIGN KEY(p0) REFERENCES points(name),FOREIGN KEY(p1) REFERENCES points(name),FOREIGN KEY(p2) REFERENCES points(name),FOREIGN KEY(p3) REFERENCES points(name))'

In [57]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

[('GEEK',),
 ('points',),
 ('segments',),
 ('between',),
 ('sameside',),
 ('parallel',),
 ('collinear',),
 ('midpoint',),
 ('congruent',),
 ('similar',),
 ('perpendicular',),
 ('concyclic',)]

In [56]:
cursor.execute("DELETE FROM parallel;")
cursor.fetchall()

[]

In [59]:
def insert_relation(relation, *points):
  cols = ",".join([f"p{i}" for i in range(len(points))])
  values = ",".join([f"'{item}'" for item in points])
  for point in points:
    query = f"INSERT OR IGNORE INTO points (name) VALUES ('{point}');"
    cursor.execute(query)
  query = f"""
  INSERT INTO {relation} ({cols})
  VALUES ({values});
  """
  cursor.execute(query)
insert_relation("parallel", "a", "b", "c", "d")

In [61]:
cursor.execute("SELECT * FROM parallel;")
cursor.fetchall()

[('a', 'b', 'c', 'd')]

In [62]:
def generate_query(formal_points, relations):
  query = "SELECT "
  for point in formal_points:
    query += f"{point}.name AS p{point}, "
  query = query[:-2] + " "
  query += "FROM "
  for point in formal_points:
    query += f"points {point}, "
  query = query[:-2]
  for i, relation in enumerate(relations):
    points = relation[1:]
    relation = relation[0]
    query += f" INNER JOIN {relation} r{i} ON "
    for j in range(len(points)):
      query += f"{points[j]}.name = r{i}.p{j} AND "
    query = query[:-5]
  cursor.execute(query)
  results = cursor.fetchall()
  return results
generate_query(["a", "b", "c", "d"], [("parallel", "a", "b", "c", "d"), ("parallel", "a", "b", "c", "d"), ("parallel", "a", "b", "c", "d")])

[('a', 'b', 'c', 'd')]